In [1]:
# load
from _1_logo_pseudo_code_generator import generateLOGOPseudoCode
from _2_sampler import LOGOProgramSampler
from _3_executable_logo_primitives import ReGALLOGOPrimitives
from _4_logo_graphic_generator_v1 import PseudoProgramInterpreter as PseudoProgramInterpreter_v1
from _4_logo_graphic_generator_v2 import PseudoProgramInterpreter as PseudoProgramInterpreter_v2
from _5_ascii_processor import ASCIIProcessor

generator=generateLOGOPseudoCode()

import json
import pandas as pd
import os

In [2]:
#LOGO
train_logo_data = "logo_data/python/train_200_dataset.jsonl"
dev_logo_data = "logo_data/python/dev_100.jsonl"
test_logo_data = "logo_data/python/test_dataset.jsonl"

# Load train, dev and test dataset
def load_data(data_path):
    with open(data_path, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

# Extract descriptions and programs from train_data and transform into pandas DataFrame
def extract_descriptions_and_programs(data):
    extracted_data = []
    for item in data:
        description = None
        program = None
        if "messages" in item:  # suits the format for train_data and test_data
            for message in item.get('messages', []):
                if message['from'] == 'human':
                    description = message['value']
                elif message['from'] == 'gpt':
                    program = message['value']
        elif "program" in item and "language" in item: # suites the format for dev_data
            program = item['program']
            description = " ".join(item['language'])

        if description and program:
            extracted_data.append([description, program])
    extracted_data = pd.DataFrame(extracted_data, columns=['Description', 'Program'])
    return extracted_data

df_train = extract_descriptions_and_programs(load_data(f"../external/dependencies/{train_logo_data}"))
df_dev = extract_descriptions_and_programs(load_data(f"../external/dependencies/{dev_logo_data}"))
df_test = extract_descriptions_and_programs(load_data(f"../external/dependencies/{test_logo_data}"))

display(df_train.tail(2))
display(df_dev.tail(2))
display(df_test.tail(2))

,Description,Program
198,8 sided snowflake with a short space and a sho...,"for j in range(8):\n embed(""""""penup()\nforw..."
199,8 sided snowflake with a medium triangle as arms,"for j in range(8):\n embed(""""""for i in rang..."


,Description,Program
98,5 sided snowflake with a short line and a smal...,"for j in range(5):\n embed(""""""forward(4)\nl..."
99,6 sided snowflake with a short space and a sho...,"for j in range(6):\n embed(""""""penup()\nforw..."


,Description,Program
109,5 sided snowflake with a small 5 gon and a sma...,"for j in range(5):\n embed(""""""penup()\nforw..."
110,6 sided snowflake with a small 5 gon and a sma...,"for j in range(6):\n embed(""""""penup()\nforw..."


In [5]:
from transform_data_to_forkstate_custom import embed_to_fork_state, transform_completion
from datasets import load_dataset

ds = load_dataset("ruthchy/semantic-length-generalization-logo-data-desc-ascii_35")

ds_transformed = ds.map(transform_completion)
print(ds_transformed)

DatasetDict({
    train: Dataset({
        features: ['Description', 'ASCII-Art', 'Program'],
        num_rows: 8006
    })
    validation: Dataset({
        features: ['Description', 'ASCII-Art', 'Program'],
        num_rows: 997
    })
    test: Dataset({
        features: ['Description', 'ASCII-Art', 'Program'],
        num_rows: 997
    })
})


In [6]:
print(ds["train"]["Program"][8:10])
print("\n")
print(ds_transformed["train"]["Program"][8:10])

['for j in range(3):\n    embed("""forward(20)""", locals())\n    forward(0)\n    left(120.0)', 'for i in range(9):\n    forward(16)\n    left(180.0 - 40.0)']


['for j in range(3):\n    with fork_state():\n        forward(20)\n    forward(0)\n    left(120.0)', 'for i in range(9):\n    forward(16)\n    left(180.0 - 40.0)']


In [8]:
print('for j in range(3):\n    embed("""forward(20)""", locals())\n    forward(0)\n    left(120.0)')

for j in range(3):
    embed("""forward(20)""", locals())
    forward(0)
    left(120.0)


In [9]:
print('for j in range(3):\n    with fork_state():\n        forward(20)\n    forward(0)\n    left(120.0)')

for j in range(3):
    with fork_state():
        forward(20)
    forward(0)
    left(120.0)


In [16]:
# Cell 1: Setup import paths
import sys
import os

# Add the root directory of external/dependencies to Python path
repo_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
dependencies_path = os.path.join(repo_root, 'external/dependencies')
sys.path.append(dependencies_path)

# Now import required modules
from program_refactoring.domains.logos.pyturtle import PyTurtle
from program_refactoring.domains.logos.utils import get_func_names
from program_refactoring.codebank import CodeBank

from datasets import load_dataset

# Load the dataset
ds = load_dataset("ruthchy/semantic-length-generalization-logo-data-desc-ascii_35")

# Get the program from validation set line 18
program = ds['validation'][18]['Program']
print("Program to execute:")
print(program)

# Create PyTurtle instance and execute program
turtle = PyTurtle()
try:
    # Execute the program
    exec(program, {'turtle': turtle, 'fork_state': turtle.embed}, {})
    
    # Save the result
    output_path = 'validation_18_output.png'
    turtle.save(output_path)
    print(f"\nOutput saved to {output_path}")
except Exception as e:
    print(f"Error executing program: {e}")

NameError: name '__file__' is not defined